## Данные
[Automobile Data Set](https://archive.ics.uci.edu/ml/datasets/Automobile). В данных присутствуют категориальные, целочисленные и вещественнозначные признаки.

In [9]:
# Внес в csv строчку с заголовками из описания датасета. Вместо целочисленных значений.
import numpy as np
import pandas as pd
%matplotlib inline
pd.set_option('display.max_columns',100)
X_raw = pd.read_csv("cars.csv").fillna('?') # Заполняем пустые значения - '?'

In [10]:
original_data = X_raw.head()
original_data

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


## Предобработка данных
Предобработка данных важна при применении любых методов машинного обучения, а в особенности для линейных моделей. В sklearn предобработку удобно делать с помощью модуля [preprocessing](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing)

In [11]:
from sklearn import preprocessing

### Преобразование нечисловых признаков
Практически все методы машинного обучения требуют, чтобы на вход функции обучения подавалась вещественная матрица. В процессе обучения используются свойства вещественных чисел, в частности, возможность сравнения и применения арифметических операций. Поэтому, даже если в матрице объекты-признаки записаны числовые значения, нужно всегда анализировать, можно ли относиться к ним как к числам. 

__Пример:__ некоторые признаки могут задаваться целочисленными хешами или id, однако нельзя сложить двух пользователей и получить третьего, исходя из их id (как это может сделать линейная модель).

Это пример категориального признака, принимающего значения из неупорядоченного конечного множества $K$. К таким признакам обычно применяют [one-hot encoding](http://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features) (вместо одного признака создают $K$ бинарных признаков - по одному на каждое возможное значение исходного признака). В sklearn это можно сделать следующим образом:

In [12]:
# для удобства работы с нашим датасетом создаем маску, указывающую на столбцы с категориальными признаками
cat_features_mask = (X_raw.dtypes == "object").values # категориальные признаки имеют тип "object"
categorical_mask = pd.DataFrame(cat_features_mask).T # Транспонируем значения с вертикального на горизонтальное
categorical_mask.columns=X_raw.columns # Добавляем заголовки из X_raw для удобства чтения
categorical_mask

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,False,True,True,True,True,True,True,True,True,False,False,False,False,False,True,True,False,True,True,True,False,True,True,False,False,True


In [13]:
# Отчистим данные от строк без знаячений
X_raw = X_raw[X_raw['normalized-losses'] != '?'] #  Убили два значения
X_raw = X_raw[X_raw['num-of-doors'] != '?'] #  Убили два значения
X_raw = X_raw[X_raw['bore'] != '?'] #  Убили четыре значения
X_raw = X_raw[X_raw['horsepower'] != '?'] #  Убили два значения
X_raw = X_raw[X_raw['price'] != '?'] #  Убили четыре значения
X_raw # минус 14 строк

# Нашли несоответсвие фичей и типов (категорийных)
# Преобразуем
for i in ['normalized-losses', 'bore', 'stroke', 'horsepower', 'peak-rpm', 'price']:
    X_raw[i] = X_raw[i].astype(float)
    
# Повторим маскирование
cat_features_mask = (X_raw.dtypes == "object").values # категориальные признаки имеют тип "object"
categorical_mask = pd.DataFrame(cat_features_mask).T # Транспонируем значения с вертикального на горизонтальное
categorical_mask.columns=X_raw.columns # Добавляем заголовки из X_raw для удобства чтения
categorical_mask

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,False,False,True,True,True,True,True,True,True,False,False,False,False,False,True,True,False,True,False,False,False,False,False,False,False,False


In [15]:
# кодирование категорий-строк натуральными числами
label_enc = preprocessing.LabelEncoder()
for feature in X_raw.columns[cat_features_mask]: 
    X_raw[feature] = label_enc.fit_transform(X_raw[feature])

In [16]:
coded_data = X_raw.head(50)
coded_data

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
3,2,164.0,0,1,0,0,3,1,0,99.8,176.6,66.2,54.3,2337,2,2,109,4,3.19,3.40,10.00,102.0,5500.0,24,30,13950.0
4,2,164.0,0,1,0,0,3,0,0,99.4,176.6,66.4,54.3,2824,2,1,136,4,3.19,3.40,8.00,115.0,5500.0,18,22,17450.0
6,1,158.0,0,1,0,0,3,1,0,105.8,192.7,71.4,55.7,2844,2,1,136,4,3.19,3.40,8.50,110.0,5500.0,19,25,17710.0
8,1,158.0,0,1,1,0,3,1,0,105.8,192.7,71.4,55.9,3086,2,1,131,4,3.13,3.40,8.30,140.0,5500.0,17,20,23875.0
10,2,192.0,1,1,0,1,3,2,0,101.2,176.8,64.8,54.3,2395,2,2,108,4,3.50,2.80,8.80,101.0,5800.0,23,29,16430.0
11,0,192.0,1,1,0,0,3,2,0,101.2,176.8,64.8,54.3,2395,2,2,108,4,3.50,2.80,8.80,101.0,5800.0,23,29,16925.0
12,0,188.0,1,1,0,1,3,2,0,101.2,176.8,64.8,54.3,2710,2,3,164,4,3.31,3.19,9.00,121.0,4250.0,21,28,20970.0
13,0,188.0,1,1,0,0,3,2,0,101.2,176.8,64.8,54.3,2765,2,3,164,4,3.31,3.19,9.00,121.0,4250.0,21,28,21105.0
18,2,121.0,2,1,0,1,2,1,0,88.4,141.1,60.3,53.2,1488,1,4,61,1,2.91,3.03,9.50,48.0,5100.0,47,53,5151.0
19,1,98.0,2,1,0,1,2,1,0,94.5,155.9,63.6,52.0,1874,2,2,90,1,3.03,3.11,9.60,70.0,5400.0,38,43,6295.0


Следует заметить, что в новой матрице очень много нулевых значений. 
Чтобы не хранить их в памяти, можно задать параметр OneHotEncoder(sparse = True), и метод fit_transform вернет [разреженную матрицу](http://docs.scipy.org/doc/scipy/reference/sparse.html), в которой хранятся только ненулевые значения. Выполнение некоторых операций с такой матрицей может быть неэффективным, однако большинство методов sklearn умеют работать с разреженными матрицами.

In [17]:
# применение one-hot encoding только для категориальных признаков в X_raw
from sklearn.preprocessing import OneHotEncoder
# encoder = OneHotEncoder(categorical_features=[x in categorical_features for x in fly_improved.columns])
# fly_encoded = ohencoder.fit_transform(fly_improved)
# fly_encoded

# Судя по домащней работе №2 следует отделить категориальные и числовые столбцы.
categorical = categorical_mask[categorical_mask == True]
categorical.dropna(axis=1,inplace=True)
numerical = categorical_mask[categorical_mask == False]
numerical.dropna(axis=1,inplace=True)

In [22]:
cat = X_raw[categorical.columns]
cat

,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,engine-type,num-of-cylinders,fuel-system
3,0,1,0,0,3,1,0,2,2,4
4,0,1,0,0,3,0,0,2,1,4
6,0,1,0,0,3,1,0,2,1,4
8,0,1,1,0,3,1,0,2,1,4
10,1,1,0,1,3,2,0,2,2,4
11,1,1,0,0,3,2,0,2,2,4
12,1,1,0,1,3,2,0,2,3,4
13,1,1,0,0,3,2,0,2,3,4
18,2,1,0,1,2,1,0,1,4,1
19,2,1,0,1,2,1,0,2,2,1


In [24]:
num = X_raw[numerical.columns]


,symboling,normalized-losses,wheel-base,length,width,height,curb-weight,engine-size,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
3,2,164.0,99.8,176.6,66.2,54.3,2337,109,3.19,3.40,10.00,102.0,5500.0,24,30,13950.0
4,2,164.0,99.4,176.6,66.4,54.3,2824,136,3.19,3.40,8.00,115.0,5500.0,18,22,17450.0
6,1,158.0,105.8,192.7,71.4,55.7,2844,136,3.19,3.40,8.50,110.0,5500.0,19,25,17710.0
8,1,158.0,105.8,192.7,71.4,55.9,3086,131,3.13,3.40,8.30,140.0,5500.0,17,20,23875.0
10,2,192.0,101.2,176.8,64.8,54.3,2395,108,3.50,2.80,8.80,101.0,5800.0,23,29,16430.0
11,0,192.0,101.2,176.8,64.8,54.3,2395,108,3.50,2.80,8.80,101.0,5800.0,23,29,16925.0
12,0,188.0,101.2,176.8,64.8,54.3,2710,164,3.31,3.19,9.00,121.0,4250.0,21,28,20970.0
13,0,188.0,101.2,176.8,64.8,54.3,2765,164,3.31,3.19,9.00,121.0,4250.0,21,28,21105.0
18,2,121.0,88.4,141.1,60.3,53.2,1488,61,2.91,3.03,9.50,48.0,5100.0,47,53,5151.0
19,1,98.0,94.5,155.9,63.6,52.0,1874,90,3.03,3.11,9.60,70.0,5400.0,38,43,6295.0


### Заполнение пропусков
В матрице объекты-признаки могут быть пропущенные значения, и это вызовет исключение при попытке передать такую матрицу в функцию обучения модели. Если пропусков немного, можно удалить объекты с пропусками из обучающей выборки. Заполнить пропуски можно разными способами:
* заполнить средними (mean, median);
* предсказывать пропущенные значения по непропущенным.

Последний вариант сложный и применяется редко. Замена пропусков средними в вещественных признаках:

In [25]:
# тут код заполнения пропусков средним .fillna()
## Не нужно делать. Поскольку данные очищены. Пропусков нет.

Всегда нужно осознавать, случайны ли пропуски в каком-то признаке. Иногда факт отсутствия информации о значении признака может сам быть важным признаком, который необходимо добавить к другим признакам.

__Пример:__ предсказание возраста пользователя по данным с его телефона. Поскольку люди старшего возраста чаще пользуются простыми телефонами, факт отсутствия каких-то данных (например, истории посещенных интернет-страниц), скорее всего, будет хорошим признаком.

Для категориальных признаков рекомендуется создавать отдельную категорию, соответствующую пропущенному значению. В наши данных пропусков в категориальных признаках нет.

### Масштабирование признаков(только числовые)
При начале работы с данными всегда рекомендуется приводить все признаки к одному масштабу.  Это важно по нескольким причинам:
* ускорение обучения модели
* улучшение численной устойчивости при работе с матрицей объекты-признаки (рядом с нулем чисел с плавающей точкой больше, чем с области больших чисел)
* для линейных моделей: интерпретация весов при признаках как меры их значимости.

Популярный способ масштабирования - нормализация: вычитание среднего из каждого признака и деление на стандартное отклонение. Реализация в sklearn (нормировать бинарные признаки не нужно):

In [26]:
def scaler(column):
    mean = column.mean() # среднее
    std = column.std()# standart deviation, стандартное отклонение, сигма
    return (column - mean) / std

for column in num.columns: 
    num[column] = scaler(num[column])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


Второй популярный способ: вычитание минимума из каждого признака, а затем деление на разницу максимального и минимального значения. Реализация в sklearn:

In [27]:
# mm_scaler = preprocessing.MinMaxScaler()
# ...... = mm_scaler.fit_transform()
# # Выдает ошибку SyntaxError: invalid syntax

Объдиняем категориальные и вещественные признаки:

In [30]:
X = pd.concat([num, cat], axis=1)


In [31]:
X.head()

,symboling,normalized-losses,wheel-base,length,width,height,curb-weight,engine-size,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,engine-type,num-of-cylinders,fuel-system
3,1.059564,1.202423,0.297218,0.363282,0.304152,0.176585,-0.257580,-0.335724,-0.411938,0.554950,-0.041428,0.200645,0.829113,-0.413638,-0.322294,0.426052,0,1,0,0,3,1,0,2,2,4
4,1.059564,1.202423,0.219810,0.363282,0.406828,0.176585,0.752917,0.550661,-0.411938,0.554950,-0.555636,0.623841,0.829113,-1.397706,-1.560840,1.021507,0,1,0,0,3,0,0,2,1,4
6,0.221401,1.034126,1.458340,1.760466,2.973718,0.793662,0.794415,0.550661,-0.411938,0.554950,-0.427084,0.461073,0.829113,-1.233695,-1.096385,1.065741,0,1,0,0,3,1,0,2,1,4
8,0.221401,1.034126,1.458340,1.760466,2.973718,0.881816,1.296551,0.386516,-0.636375,0.554950,-0.478505,1.437681,0.829113,-1.561717,-1.870477,2.114592,0,1,1,0,3,1,0,2,1,4
10,1.059564,1.987808,0.568146,0.380638,-0.414577,0.176585,-0.137233,-0.368553,0.747653,-1.479723,-0.349953,0.168091,1.473229,-0.577650,-0.477113,0.847974,1,1,0,1,3,2,0,2,2,4


In [33]:
# Применим OneHoteEncoding только для категориальных признаков
from sklearn.preprocessing import OneHotEncoder
ohencoder = OneHotEncoder(categorical_features=[x in cat.columns for x in X.columns])
X_encoded = ohencoder.fit_transform(X)
X_encoded

<159x65 sparse matrix of type '<class 'numpy.float64'>'
	with 4134 stored elements in COOrdinate format>